# 1.马老师考试的代码说明
## 这是我根据自己的想法再结合马老师课上讲的内容，写了个通过乔利斯分解并采用向后增加变量来实现最优子集回归算法的实现。当时在课堂上我思路想错了，导致没有写出来，完全错误，课下仔细思索了一番，发现原来还能这样。当然其中还有一些不太成熟的地方，和老师的代码也不会相同。欢迎各位同学猛烈拍砖，提出建议，多多交流哈！废话不多说，先来一看这到底是个什么东西吧。（以下内容可能引起不适，建议找一个开心的时候再来研究哈！）
###################################
### 先说一下总体思路，实质上就是老师课上画的那个节点图，举个简单的例子，比如总共只有3个变量，那么就应该回归7个方程，即是3的全部组合数（注意不是排列数），然后就是通过乔利斯分解解方程得到的参数就是回归系数，这个在逐步回归里已经说过啦。现在关键是在于如何构造所有的组合，我的思路就是先从一个变量开始，算出每一个下三角矩阵，存在列表里，然后再遍历两个变量的情况，并将下三角矩阵进行追加在该列表中，依次类推，最后通过比较所有回归的AIC，选出其中最小的作为最优的回归结果。说了可能还是一脸懵，不多说，先上代码！（这里面我遇到了很多坑，比如如何取出一串数字最后一位，我思索了很久，才发现R中竟然可以这样用，才发现还是Python大法好用啊，一个【:-1】就能解决，具体可看后面，大家没事也可以想一想。）



In [2]:
 ##这一部分是上节课的内容，大家应该没问题了
###第一部分，乔利斯分解###
 Cholesky = function(Matr)
 {if(length(Matr)==1){
   m = 1
   n = 1
   if (m!=n) return ('the Matrix_dim is wrong!')
   L=matrix(0,m,m)
   for(i in 1:m)
   {
     L[i,i]=sqrt(Matr)
     if(i<m)
     {
       L[(i+1):m,i]=Matr[(i+1):m,i]/L[i,i]
       TL=L[(i+1):m,i]
       TM=outer(TL,TL)
       Matr[(i+1):m,(i+1):m]=Matr[(i+1):m,(i+1):m]-TM
     }
   }
   L
 }
   else{
     m = dim(Matr)[1]
     n = dim(Matr)[2]
     if (m!=n) return ('the Matrix_dim is wrong!')
     L=matrix(0,m,m)
     for(i in 1:m)
     {
       L[i,i]=sqrt(Matr[i,i])
       if(i<m)
       {
         L[(i+1):m,i]=Matr[(i+1):m,i]/L[i,i]
         TL=L[(i+1):m,i]
         TM=outer(TL,TL)
         Matr[(i+1):m,(i+1):m]=Matr[(i+1):m,(i+1):m]-TM
       }
     }
     L
   }
   
 }
 
 ###前代法
 prefoward=function(L,B)
 {if(length(L)==1){
   m = 1
   n = 1
   if(m!=n) return ('the dim of Matrix is Wrong')
   if(m!=length(B)) return('Wrong dimensions of matrix L or vector b! Please check')
   x = rep(0,m)
   for (i in 1:m){
     x[i]=B[i]/L
     if(i<m){
       B[(i+1):m]= B[(i+1):m]-x[i]*L[(i+1):m,i]
     }
   }
   x
 }else{
   m=dim(L)[1]
   n=dim(L)[2]
   if(m!=n) return ('the dim of Matrix is Wrong')
   if(m!=length(B)) return('Wrong dimensions of matrix L or vector b! Please check')
   x = rep(0,m)
   for (i in 1:m){
     x[i]=B[i]/L[i,i]
     if(i<m){
       B[(i+1):m]= B[(i+1):m]-x[i]*L[(i+1):m,i]
     }
   }
   x
 }
   
 }
 
 ###回代发
 backfoward = function(L,B)
 {
   m=dim(L)[1]
   n=dim(L)[2]
   if(n!=length(B)) return('Wrong dimensions of matrix L or vector b! Please check')
   if(m!=n) return ('the dim of Matrix is Wrong')
   x = rep(0,m)
   for (i in m:1){
     x[i] = B[i]/L[i,i]
     if(i>1){
       B[(i-1):1]=B[(i-1):1]-x[i]*L[(i-1):1,i]
     }
   }
   x
 }
 ###增加一个变量
 forupdate = function(L,xxk,xkxk){
   lk = prefoward(L,xxk)
   lkk = sqrt(xkxk-sum(lk*lk))
   return (as.matrix(rbind(cbind(L,0),c(lk,lkk))))
 }
 

In [91]:
###导入数据
 library(ElemStatLearn)                     #加载ElemStatLearn包
 data(prostate)                             #加载prostate数据集
 data = prostate  
 # print(data)
 p= dim(data)[2]-2                         #计算自变量个数 p = 8
 datatr = data[data[,p+2],1:(p+1)]          #获得训练集  
 # datate = data[!data[,p+2],1:(p+1)]         #测试集
 # datatr
 p = p-4  #以4个变量为例,因为总共8个变量
 xn = c('inter',names(datatr)[1:p])
 #     return (xn)
 x = as.matrix(cbind(1,datatr[,1:p]))
 y = datatr[,p+1]
 
 #########数据中心化
 # x = scale(x,center = T,scale = F)
 # y = scale(y ,center = T,scale = F)
 
 # print(datatr)
 xtx = t(x)%*%x
 xty = t(x)%*%y
 yty = t(y)%*%y

 L =Cholesky(xtx)
 n = dim(datatr)[1]
 L_list = list(rep(0,(2^p-1)))
 
 bm = matrix(0,(2^p-1),p)
 AICm = rep(0,(2^p-1))
 tA  = p-1


In [92]:
####这是核心代码####
###主要是对每一个组合求出其下三角矩阵，但是每次增加一个变量可在上一层循环中，使用forupdate函数进行增加，而无需再重复计算乔利斯分解，这是
##这个方法提升了不少，里面有三重循环，结合节点图的例子，慢慢推一下，就比较好理解。命名比较暴力，直接ABCD了
comb = function(p,L,list1){
   A = list()
   for (i in 1:p){
     A = c(A,list(i))
     list1[[i]] = L[A[[i]],A[[i]]]
   }
   B = A
   D = A
   while (length(B)>1){
     C = list()
     for (i in (1:length(B))){
       for (j in (1:length(A))){
         if (length(B[[i]][-1])!=0){
           if (A[[j]]>B[[i]][length(B[[i]])]){##判断是否比最后一个数字大，这里大家可以仔细考虑一下怎么实现
             M = B[[i]]
             LA = Cholesky(xtx[M,M])
             xxk = xtx[M,j,drop =T]
             xkxk =xtx[j,j]
             # LA = L[B[[i]],B[[i]]]
             Lk = forupdate(LA,xxk,xkxk)
             list1 =c(list1,list(Lk))
             C = c(C,list(c(M,A[[j]])))
           }
         }else{
           if (A[[j]]>B[[i]]){
             M = B[[i]]
             LA = Cholesky(xtx[M,M])
             xxk = xtx[B[[i]],j,drop =T]
             xkxk =xtx[j,j]
             # LA = L[B[[i]],B[[i]]]
             Lk = forupdate(LA,xxk,xkxk)
             list1 =c(list1,list(Lk))
             C = c(C,list(c(B[[i]],A[[j]])))
           }
         }
       }
     }
     B = C
     D = c(D,C)
     
   }
   # D
   c(list1,D)
   # D
 }

In [93]:
ComB = comb(p,L,L_list)
L_list = ComB[1:(2^p-1)]
FLAG =  ComB[2^p:length(ComB)]
for (i in 1:(2^p-1)){
  flag = FLAG[[i]]
  xtxk = xtx[flag,flag]
  xtyk = xty[flag,1]
  yty = t(y)%*%y
  ###运行结果p
  Lk = L_list[[i]]
  tbk = prefoward(Lk,xtyk)
  bk = backfoward(t(Lk),tbk)
  #####
  bm[i,flag] = bk
  #计算RSSK
  RSSk = yty - sum(tbk*tbk)
  AICk = n*log(RSSk/n) + 2*length(p-1)
  AICm[i] = AICk
}
##
bm


Warning message in log(RSSk/n):
"产生了NaNs"
Warning message in log(RSSk/n):
"产生了NaNs"
Warning message in log(RSSk/n):
"产生了NaNs"


0.22388060,0.0000000,0.000000000,0.0000000000
0.00000000,0.3937523,0.000000000,0.0000000000
0.00000000,0.0000000,4.162881664,0.0000000000
0.00000000,0.0000000,0.000000000,0.3125835165
-0.03936109,0.2004137,0.000000000,0.0000000000
-0.35465911,0.0000000,0.159548407,0.0000000000
-0.24334228,0.0000000,0.000000000,0.0072162132
0.00000000,0.2009644,-0.010831931,0.0000000000
0.00000000,0.2028649,0.000000000,-0.0006804552
0.00000000,0.0000000,0.074585576,-0.0006353482
-0.04956579,0.2000755,0.002936727,0.0000000000


In [94]:
###输出最优的子集结果
# which.min(AICm)
print("最优的回归结果是：" )
print(names(data)[FLAG[[which.min(AICm)]]])
print("回归系数：" )
print(bm[which.min(AICm),])

[1] "最优的回归结果是："
[1] "lcavol"  "lweight" "age"     "lbph"   
[1] "回归系数："
[1]  0.082262058  0.203517376  0.013765088 -0.002712334
